# When should `AwkwardInputLayer`'s number of partitions be updated?

We have an `AwkwardInputLayer` that has 10 partitions - but we don't know that till late in the game. So lets start with a single partition, and then update it when the user calls `.compute()` on the file.

In [1]:
import dask_awkward as dak
import awkward as ak
import dask
import uproot

## Input Layer

The function that will generate the data - it is dead simple as it always returns the same data (but _pretend_!!!)

In [2]:
def generate_data(i_partition: int) -> ak.Array:
    '''Generate the data for a particular partition. There is always one
    file per partition.

    Use a file that is from an actual SX query, and has "JetPt' in it.

    Args:
        block (int): The block number
    '''
    print(f"Loading data for partition {i_partition}")
    filename = "0fc6e51a5ea6dea107c195591d20a1b2-15.26710677._000019.pool.root.1"
    pt = uproot.open(filename)['treeme'].arrays(library='ak')  # type: ignore
    return pt

And now we can create the input layer. THis is the "dummy" version - with only 1 partition when we create, but want to come back during the `compute()` call and update to be 10.

In [3]:
name_layer = "SX_Query_Result"

input_layer = dak.layers.AwkwardInputLayer(
    name=name_layer,
    inputs=[0],
    io_func=generate_data,
)

Before creating the high level array, we need the metadata/type info that describes the Schema.

In [4]:
sample_array = ak.from_iter([
    {"JetPt": [1.0, 2.1]},
])
type_info = dak.core.typetracer_array(sample_array)
type_info

<Array-typetracer [...] type='## * {JetPt: var * float64}'>

Now we can create the high level array that we can then "query" against.

In [5]:
hlg = dask.highlevelgraph.HighLevelGraph.from_collections(name_layer, input_layer)
ar = dak.core.new_array_object(hlg, name_layer, meta=type_info, npartitions=1)

## Build the query and run

When we run this, we want it to print out:

```text
Loading data for partition 0
Loading data for partition 1
Loading data for partition 2
Loading data for partition 3
Loading data for partition 4
Loading data for partition 5
Loading data for partition 6
Loading data for partition 7
Loading data for partition 8
Loading data for partition 9

```

In [6]:
pt = ar.JetPt * 5
pt.compute()

Loading data for partition 0


<Array [[509, 315], [...], ..., [763, 452, 270]] type='50000 * var * float64'>

In [7]:
pt

dask.awkward<multiply, npartitions=1>

## Doing it by hand

Try changing the `.inputs` by hand (`ar.npartitions` can't be set it seems)

In [8]:
input_layer.inputs = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
pt

dask.awkward<multiply, npartitions=1>

In [9]:
pt.compute()

Loading data for partition 0


<Array [[509, 315], [...], ..., [763, 452, 270]] type='50000 * var * float64'>